<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/5_Aprendizaje_supervisado/5_Ensamble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# 🥦🤝 **Métodos de Ensamble: Random Forest y Gradient Boosting**

**Objetivos de Aprendizaje**

Al finalizar este notebook, serás capaz de:

1.  **Explicar la intuición** detrás de los métodos de ensamble como Random Forest (Bagging) y Gradient Boosting (Boosting).
2.  **Implementar y comparar** modelos de ensamble con un modelo base (Árbol de Decisión) utilizando `scikit-learn`.
3.  **Aplicar** el uso de métricas avanzadas como la Curva ROC y el AUC en contextos con clases desbalanceadas, como el riesgo crediticio.
4. **Demostrar** de forma rápida y directa cómo cambia el rendimiento de cada modelo en el set de prueba.
5.  **Extraer** la importancia de las variables (`feature importance`).

**Introducción: La Sabiduría del Comité**

Hasta ahora, hemos entrenado modelos de forma individual. Un árbol de decisión, una regresión logística, una regresión lineal, etc. Pero, ¿qué pasaría si en lugar de confiar en la opinión de un solo "experto" (un modelo), creamos un comité de expertos y basamos nuestra decisión en su sabiduría colectiva?

Esta es la idea central detrás de los **métodos de ensamble**. Combinan múltiples modelos, a menudo llamados "aprendices débiles", para crear un "aprendiz fuerte" mucho más robusto, preciso y menos propenso al sobreajuste.

**Analogía**

* **Random Forest (Bagging):** Imagina un **comité de inversión diverso**. En lugar de confiar en un solo analista estrella (un único árbol de decisión), el portafolio se beneficia de las opiniones diversas y no correlacionadas de muchos analistas. Cada analista recibe una muestra ligeramente diferente de la información disponible. Incluso si algunos se equivocan, el **promedio del comité es mucho más estable y acertado**.

* **Gradient Boosting (Boosting):** Piensa en un **programa de mentoría para analistas de riesgo**. El primer analista (modelo 1) evalúa una solicitud de crédito y comete algunos errores. Un analista senior o mentor (modelo 2) revisa esos errores y aprende a identificarlos. Luego, un tercer analista (modelo 3) se enfoca en los errores que cometieron los dos primeros. Este proceso secuencial de **aprender de los errores anteriores** crea un pronóstico final muy refinado y preciso.

**IMPORTANTE:**

- Ambos resuelven tanto problemas de regresión como de clasificación (vamos a ver un ejemplo de clasificación y en el taller van a trabajar un ejemplo de regresión).

## 1. Preparación del Entorno y Datos

In [ ]:
# Importación de librerías estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Módulos de Scikit-Learn para modelado y evaluación
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve

### Mejorar visualización de dataframes y gráficos

In [ ]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# Configuraciones para una mejor visualización
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### Carga y Limpieza de Datos

Usaremos el dataset **German Credit Data** (https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data), el cual contiene información sobre 1000 solicitantes de crédito. El conjunto de datos incluye a individuos a quienes efectivamente se les otorgó un crédito. No contiene información sobre los solicitantes que fueron rechazados.

Para cada uno de estos 1000 individuos, el banco ya realizó una clasificación de riesgo a posteriori, es decir, después de observar su comportamiento de pago. **La columna objetivo es Risk (o 'Riesgo')** indica si, con el tiempo, el cliente resultó ser un buen pagador (cumplió con sus obligaciones) o un mal pagador (incurrió en impago o default).

Por lo tanto, el objetivo de un modelo entrenado con estos datos no es decidir si se aprueba o no un crédito (ya que todos fueron aprobados), sino predecir, en el momento de la solicitud, la probabilidad de que un cliente aprobado se convierta en un mal pagador en el futuro.

El dataset original de la UCI no tiene nombres de columna y las variables categóricas necesitan ser procesadas.

In [ ]:
# Cargar el dataset desde una URL pública
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data'
# Los nombres de las columnas se encuentran en la documentación del dataset
columns = ['Estado de la cuenta corriente existente', 'Duración en meses', 'Historia de crédito', 'Propósito',
           'Monto del crédito',
           'Cuenta de ahorros/bonos', 'Empleo actual desde',
           'Tasa de pago a plazos en porcentaje del ingreso disponible',
           'Estado personal y sexo', 'Otros deudores/fiadores', 'Residencia actual desde', 'Propiedad',
           'Edad', 'Otros planes de pago', 'Alojamiento', 'Número de créditos existentes en este banco',
           'Trabajo', 'Número de personas obligadas a prestar manutención a', 'Teléfono', 'Trabajador extranjero',
           'Riesgo']

df = pd.read_csv(url, sep=' ', header=None, names=columns)

# Vistazo inicial a los datos
print("Dimensiones del dataset:", df.shape)
df.head()

In [ ]:
# Cargar el dataset desde una URL pública
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data'
# Los nombres de las columnas se encuentran en la documentación del dataset
columns = ['Estado de la cuenta corriente existente', 'Duración en meses', 'Historia de crédito', 'Propósito',
           'Monto del crédito',
           'Cuenta de ahorros/bonos', 'Empleo actual desde',
           'Tasa de pago a plazos en porcentaje del ingreso disponible',
           'Estado personal y sexo', 'Otros deudores/fiadores', 'Residencia actual desde', 'Propiedad',
           'Edad', 'Otros planes de pago', 'Alojamiento', 'Número de créditos existentes en este banco',
           'Trabajo', 'Número de personas obligadas a prestar manutención a', 'Teléfono', 'Trabajador extranjero',
           'Riesgo']

df = pd.read_csv(url, sep=' ', header=None, names=columns)

# Vistazo inicial a los datos
print("Dimensiones del dataset:", df.shape)
df.head()

In [ ]:
# --- 1. Definición de los Diccionarios de Mapeo ---

# Atributo 1: Estado de la cuenta corriente existente
estado_cuenta_corriente_map = {
    'A11': 'Menos de 0 DM',
    'A12': 'Entre 0 y 200 DM',
    'A13': 'Más de 200 DM / Salario asignado',
    'A14': 'Sin cuenta corriente'
}

# Atributo 3: Historial crediticio
historial_crediticio_map = {
    'A30': 'Sin créditos / Todos pagados',
    'A31': 'Todos los créditos en este banco pagados',
    'A32': 'Créditos existentes pagados hasta ahora',
    'A33': 'Retraso en pagos en el pasado',
    'A34': 'Cuenta crítica / Otros créditos'
}

# Atributo 4: Propósito del crédito
proposito_map = {
    'A40': 'Automóvil (nuevo)',
    'A41': 'Automóvil (usado)',
    'A42': 'Muebles / Equipo',
    'A43': 'Radio / Televisión',
    'A44': 'Electrodomésticos',
    'A45': 'Reparaciones',
    'A46': 'Educación',
    'A48': 'Reciclaje',
    'A49': 'Negocios',
    'A410': 'Otros'
}

# Atributo 6: Cuenta de ahorros/bonos
cuenta_ahorros_map = {
    'A61': 'Menos de 100 DM',
    'A62': 'Entre 100 y 500 DM',
    'A63': 'Entre 500 y 1000 DM',
    'A64': 'Más de 1000 DM',
    'A65': 'Desconocido / Sin cuenta'
}

# Atributo 7: Empleo actual desde
empleo_actual_map = {
    'A71': 'Desempleado',
    'A72': 'Menos de 1 año',
    'A73': 'Entre 1 y 4 años',
    'A74': 'Entre 4 y 7 años',
    'A75': 'Más de 7 años'
}

# Atributo 9: Estado civil y sexo
estado_civil_sexo_map = {
    'A91': 'Hombre: divorciado/separado',
    'A92': 'Mujer: divorciada/separada/casada',
    'A93': 'Hombre: soltero',
    'A94': 'Hombre: casado/viudo',
    'A95': 'Mujer: soltera'
}

# Atributo 10: Otros deudores/fiadores
otros_deudores_map = {
    'A101': 'Ninguno',
    'A102': 'Co-solicitante',
    'A103': 'Garante'
}

# Atributo 12: Propiedad
propiedad_map = {
    'A121': 'Bienes inmuebles',
    'A122': 'Seguro de vida / Ahorro para vivienda',
    'A123': 'Automóvil u otro',
    'A124': 'Desconocido / Sin propiedad'
}

# Atributo 14: Otros planes de pago
otros_planes_pago_map = {
    'A141': 'Banco',
    'A142': 'Tiendas',
    'A143': 'Ninguno'
}

# Atributo 15: Alojamiento
vivienda_map = {
    'A151': 'Alquiler',
    'A152': 'Propia',
    'A153': 'Gratuita'
}

# Atributo 17: Trabajo
empleo_map = {
    'A171': 'Desempleado / No cualificado - no residente',
    'A172': 'No cualificado - residente',
    'A173': 'Empleado cualificado / Funcionario',
    'A174': 'Directivo / Autónomo / Altamente cualificado'
}

# Atributo 19: Teléfono
telefono_map = {
    'A191': 'No tiene',
    'A192': 'Sí, a su nombre'
}

# Atributo 20: trabajador extranjero
trabajador_extranjero_map = {
    'A201': 'Sí',
    'A202': 'No'
}

# --- 2. Aplicación de los Mapeos al DataFrame ---
# Se aplica el mapeo a cada columna usando los nombres en español
df['Estado de la cuenta corriente existente'] = df['Estado de la cuenta corriente existente'].map(estado_cuenta_corriente_map)
df['Historia de crédito'] = df['Historia de crédito'].map(historial_crediticio_map)
df['Propósito'] = df['Propósito'].map(proposito_map)
df['Cuenta de ahorros/bonos'] = df['Cuenta de ahorros/bonos'].map(cuenta_ahorros_map)
df['Empleo actual desde'] = df['Empleo actual desde'].map(empleo_actual_map)
df['Estado personal y sexo'] = df['Estado personal y sexo'].map(estado_civil_sexo_map)
df['Otros deudores/fiadores'] = df['Otros deudores/fiadores'].map(otros_deudores_map)
df['Propiedad'] = df['Propiedad'].map(propiedad_map)
df['Otros planes de pago'] = df['Otros planes de pago'].map(otros_planes_pago_map)
df['Alojamiento'] = df['Alojamiento'].map(vivienda_map)
df['Trabajo'] = df['Trabajo'].map(empleo_map)
df['Teléfono'] = df['Teléfono'].map(telefono_map)
df['Trabajador extranjero'] = df['Trabajador extranjero'].map(trabajador_extranjero_map)

# --- 3. Verificación de los Resultados ---
print("DataFrame con valores reemplazados y columnas en español:")
display(df.head(3))

In [ ]:
df.info()

La variable objetivo 'riesgo' está codificada como 1 (Bueno) y 2 (Malo). La convertiremos a 0 (Bueno) y 1 (Malo) para que la clase "positiva" sea el evento de interés (el impago).

- **0 = Buen pagador**
- **1 = Mal pagador**

In [ ]:
# Usamos .map() para la recodificación
df['Riesgo'] = df['Riesgo'].map({1: 0, 2: 1})

print("Distribución de la variable objetivo 'Riesgo':")
print(df['Riesgo'].value_counts(normalize=True))

Vemos un desbalance: 70% de los clientes son buenos pagadores y 30% son malos pagadores. Este **desbalance de clases**, aunque no es extremo, ya nos indica que la simple precisión (accuracy) no es la mejor métrica.

## Preprocesamiento

Vamos a convertir variables categóricas en numéricas usando One-Hot Encoding

In [ ]:
# --- PASO 1: Separar X e y ---
# X contiene TODAS las variables predictoras, tanto numéricas como categóricas
X = df.drop('Riesgo', axis=1)
y = df['Riesgo']

# --- PASO 2: División en train y test ---
# Esto es crucial para evitar data leakage. El encoder solo debe "ver" los datos de entrenamiento.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Forma de X_train original:", X_train.shape)
print("Forma de X_test original:", X_test.shape)

# --- PASO 3: Identificar columnas y crear el preprocesador ---

# Identificamos automáticamente las columnas numéricas y categóricas desde X_train
numerical_features = X_train.select_dtypes(include=np.number).columns
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns

# Crear el transformador para las variables categóricas
categorical_transformer = OneHotEncoder(handle_unknown='ignore', drop='first')

# Crear el ColumnTransformer
# 'passthrough' indica que las columnas numéricas no deben ser modificadas
# En árboles de decisión, random forest o boosting no es necesario estandarizar las variables numéricas,
# pero hacerlo, tampoco es perjudicial
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', 'passthrough', numerical_features)
    ],
    remainder='passthrough' # Por si alguna columna no fue listada, se le indica que no le haga nada
)

# --- PASO 4: Aplicar el preprocesador ---

# "Aprender" (fit) las categorías SOLO de X_train y transformar X_train
X_train = preprocessor.fit_transform(X_train)

# Usar el preprocesador ya "aprendido" para transformar X_test
X_test = preprocessor.transform(X_test)

# --- Verificación de resultados ---
print("\nForma de X_train procesado:", X_train.shape)
print("Forma de X_test procesado:", X_test.shape)

## 2. Modelo Base: Un Solo Árbol de Decisión

Antes de construir un "bosque", entendamos el rendimiento de un solo "árbol". Este será nuestro punto de comparación para ver si la complejidad de los ensambles realmente aporta valor.

In [ ]:
# Inicializar y entrenar el Árbol de Decisión
tree_model = DecisionTreeClassifier(random_state=42, max_depth=20) # Limitamos la profundidad para evitar sobreajuste extremo
tree_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_tree = tree_model.predict(X_test)

# Visualizar la matriz de confusión
cm_tree = confusion_matrix(y_test, y_pred_tree)
sns.heatmap(cm_tree, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicción 0 (Bueno)', 'Predicción 1 (Malo)'],
            yticklabels=['Real 0 (Bueno)', 'Real 1 (Malo)'])
plt.ylabel('Valor Real')
plt.xlabel('Valor Predicho')
plt.title('Matriz de Confusión Árbol de Decisión')
plt.show()

# Evaluar el modelo
print("Evaluación del Árbol de Decisión:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_tree):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_tree))

**Interpretación:**

El árbol único tiene una `exactitud` del 64%. Sin embargo, miremos más de cerca el `recall` para la clase 1 (mal pagador): es solo de 0.42. Esto significa que **nuestro modelo solo está identificando al 42% de los clientes que realmente van a incumplir**. Para un banco, ¡esto es muy riesgoso! Estamos dejando pasar a más de la mitad de los malos clientes.

La `precisión` también es baja. De los que el modelo predijo que eran malos pagadores, solo 40% son realmente malos pagadores.



## 3. Random Forest: La Sabiduría de la Multitud 🌳🌳🌳

Hasta ahora, hemos usado un solo Árbol de Decisión. Siguiendo nuestra analogía, esto es como confiar en el juicio de un único "analista estrella". ¿Pero qué pasa si ese analista tiene un sesgo particular o se ha sobreajustado a una mala racha del mercado? Su modelo será inestable y poco fiable.

**Random Forest (Bosque Aleatorio)** propone una solución mucho más robusta, basada en el principio de "la sabiduría de la multitud".

En lugar de confiar en un solo analista, vamos a crear un comité de inversión diverso con, por ejemplo, 300 analistas (árboles). La decisión final no la tomará uno solo, sino que será el promedio o la votación de todo el comité.

Para que este comité funcione, necesitamos dos cosas:

- Competencia: Cada analista (árbol) debe ser razonablemente bueno (mejor que adivinar al azar).

- Diversidad: Los analistas no deben cometer los mismos errores. Sus opiniones deben ser no correlacionadas. Si todos piensan igual, el comité es inútil.

Random Forest logra esta diversidad de forma brillante usando dos técnicas de aleatorización:

1. **Aleatorización de Datos (Bagging)**

No le damos a todos los analistas exactamente los mismos datos históricos. En lugar de eso, a cada uno de los 300 árboles le damos una muestra aleatoria del conjunto de entrenamiento.

Esta técnica se llama Bagging. Para un dataset de 800 clientes de entrenamiento, cada árbol recibirá una muestra de 800 clientes elegidos con reemplazo. Esto significa que la muestra de cada árbol será ligeramente diferente: algunas contendrán clientes repetidos y otras omitirán algunos clientes por completo.

Esto asegura que cada árbol tenga una "visión" del mundo ligeramente distinta, evitando que todos aprendan exactamente los mismos patrones.

2. **Aleatorización de Variables (Features)**

Si hay una variable muy predictiva (ej. historial_credito), todos los árboles tenderían a usarla como su primera pregunta. Esto haría que todos los árboles fueran muy similares y sus errores estarían correlacionados.

Para evitar esto, Random Forest añade una segunda capa de aleatoriedad: en cada paso de división de un árbol, no le permite ver todas las variables. Solo le da un subconjunto aleatorio de ellas (ej. "solo puedes elegir entre 'edad', 'propósito' y 'ahorros' para esta decisión").

Esto fuerza a los árboles a explorar otras variables y encontrar patrones diferentes. Un árbol podría descubrir un patrón en la 'edad' y otro en el 'monto_credito', haciendo que el comité en su conjunto sea mucho más diverso y sabio.

**¿Cómo Funciona el Modelo?**

- Se construyen n_estimators (ej. 300 árboles de decisión).

  - Cada árbol se entrena con una muestra (Bagging).
  - En cada división de cada árbol, solo se considera un subconjunto aleatorio de max_features (variables).

- Predicción (Votación):

  - Para un nuevo cliente, se le pasa su información a los 300 árboles.
  - Cada árbol da su predicción (ej. 70 árboles votan "Riesgo 0" y 30 árboles votan "Riesgo 1").
  - La predicción final del bosque es la clase más votada ("Riesgo 0"). Si es un problema de regresión, entonces la predicción final corresponde al promedio.

**La ventaja principal del Random Forest es que reduce el sobreajuste (overfitting).** Los errores de un árbol individual (que se sobreajusta) se promedian y cancelan con los errores de los demás. El "promedio del comité" es mucho más estable que un solo árbol de decisión.


Ahora, en lugar de un solo árbol, vamos a crear un bosque de 300 árboles.

In [ ]:
# Inicializar y entrenar el Random Forest
rf_model = RandomForestClassifier(n_estimators=300, random_state=42, max_depth=20, n_jobs=-1) # n_jobs=-1 usa todos los procesadores del computador
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_rf = rf_model.predict(X_test)

# Visualizar la matriz de confusión
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicción 0 (Bueno)', 'Predicción 1 (Malo)'],
            yticklabels=['Real 0 (Bueno)', 'Real 1 (Malo)'])
plt.ylabel('Valor Real')
plt.xlabel('Valor Predicho')
plt.title('Matriz de Confusión Random Forest')
plt.show()

# Evaluar el modelo
print("Evaluación del Random Forest:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

**Interpretación:** ¡Mejoramos! La `exactitud` subió al 78%. Más importante aún, el `recall` para la clase 1 (mal pagador) mejoró a 45% y la `precisión` subió de forma importante a 71%.

## 4. Gradient Boosting: Aprendiendo de los Errores 👨‍🏫

Si Random Forest es un "comité democrático" que vota en paralelo, Gradient Boosting es un "programa de mentoría" jerárquico que aprende de forma secuencial.

La intuición central de Gradient Boosting es aprender de los errores. En lugar de construir 300 modelos independientes, construimos un modelo a la vez, donde cada nuevo modelo se dedica a corregir los errores que cometió el anterior.

1. Se entrena un primer árbol de decisión simple (ej. max_depth=3). Este árbol hace una predicción inicial sobre la variable objetivo (ser buen o mal pagador en este caso).

2. Se calcula el error que este árbol comete para cada cliente (Error1 = Dato Real - Predicción(Árbol 1)).

3. Se entrena un segundo árbol, cuya variable objetivo es el error que cometió el Árbol 1 (Error1). Las variables (features) son las mismas del modelo original (Edad, Deuda, ...).

4. Se calcula una nueva predicción: Predicción combinada = Predicción(Árbol 1) + Predicción(Árbol 2). De esta forma, la predicción se va acercando más al valor real.

5. Se calculan los errores de la predicción combinada (Error2 = Dato Real - Predicción Combinada).

6. Se entrena un tercer árbol cuya variable objetivo es el error de la predicción combinada (Error2).

Este proceso se repite 'n_estimators' veces. Cada nuevo árbol se enfoca en los casos más difíciles que el "comité" anterior no pudo resolver, refinando la predicción paso a paso hasta que los errores son mínimos.

El Hiperparámetro Clave:

**Tasa de Aprendizaje (learning_rate):** controla la magnitud de la corrección.

En lugar de sumar la predicción completa:

Predicción combinada = Predicción(Árbol 1) + Predicción(Árbol 2)

Se hace una corrección más cautelosa:

Predicción combinada = Predicción(Árbol 1) + learning_rate * Predicción(Árbol 2)

Un 'learning_rate' bajo significa que cada árbol hace una corrección pequeña y sutil. Esto evita que el modelo corrija en exceso (sobreajuste) y le permite encontrar un resultado óptimo de forma más estable.

**La principal ventaja de Gradient Boosting es que al enfocarse secuencialmente en los errores, puede alcanzar un rendimiento predictivo extremadamente alto.** Suele ser uno de los mejores algoritmos para datos tabulares (como los que usamos en finanzas y economía).

La implementación más famosa y exitosa de este método es **XGBoost (Extreme Gradient Boosting)**, conocida por su eficiencia y por ganar innumerables competencias de ciencia de datos.

Desventajas:

- Requiere un ajuste cuidadoso de hiperparámetros para evitar el sobreajuste.

- A diferencia de Random Forest, no se puede paralelizar (n_jobs=-1 no sirve de mucho), ya que cada árbol depende del resultado del anterior.



El árbol de decisión y el random forest tenían una profundidad de 20, sin embargo, en Gradient Boosting se utilizan modelos sencillos (llamados 'débiles'), que son poco profundos, así que vamos a utilizar una profundidad de 3. Adicionalmente, vamos a decirle que construya 100 árboles (o que itere 100 veces).

In [ ]:
# Inicializar y entrenar el Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_gb = gb_model.predict(X_test)

# Visualizar la matriz de confusión
cm_gb = confusion_matrix(y_test, y_pred_gb)
sns.heatmap(cm_gb, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicción 0 (Bueno)', 'Predicción 1 (Malo)'],
            yticklabels=['Real 0 (Bueno)', 'Real 1 (Malo)'])
plt.ylabel('Valor Real')
plt.xlabel('Valor Predicho')
plt.title('Matriz de Confusión Árbol de Decisión')
plt.show()

# Evaluar el modelo
print("Evaluación de Gradient Boosting:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_gb))

**Interpretación:** Los resultados son aún mejores. La `exactitud` es del 80% y el `recall` para la clase 1 (mal riesgo) subió a **0.57**. ¡Ahora estamos identificando al 57% de los malos clientes! La `precisión` bajó de 0.71 a 0.69, pero el f1-score subió de 0,7 a 0.74.

## 5. Métricas de Desempeño

Como vimos, la exactitud (accuracy) no nos cuenta toda la historia, especialmente con clases desbalanceadas. Un banco podría preferir un modelo que identifique a más clientes de alto riesgo (mayor `recall`), incluso si eso significa clasificar erróneamente a algunos buenos clientes como malos (menor `precision`).

La **Curva ROC (Receiver Operating Characteristic)** nos ayuda a visualizar este trade-off.

**¿Qué es la Curva ROC?**

* **Eje Y (Tasa de Verdaderos Positivos / Recall):** ¿Qué proporción de los malos clientes reales identificamos correctamente?
* **Eje X (Tasa de Falsos Positivos):** ¿Qué proporción de los buenos clientes identificamos incorrectamente como malos?

Un modelo ideal estaría en la esquina superior izquierda (100% de Verdaderos Positivos, 0% de Falsos Positivos). La línea diagonal representa un modelo que adivina al azar. Queremos que nuestra curva esté lo más lejos posible de esa línea.

El **Área Bajo la Curva (AUC)** resume esta gráfica en un solo número. Un AUC de 0.5 es azar y un AUC de 1.0 es un modelo perfecto. Mide la habilidad del modelo para distinguir entre un cliente bueno y uno malo.

In [ ]:
# Obtener las probabilidades de predicción para la clase positiva (riesgo=1)
y_prob_tree = tree_model.predict_proba(X_test)[:, 1]
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]
y_prob_gb = gb_model.predict_proba(X_test)[:, 1]

# Calcular FPR, TPR y AUC para cada modelo
fpr_tree, tpr_tree, _ = roc_curve(y_test, y_prob_tree)
auc_tree = auc(fpr_tree, tpr_tree)

fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)
auc_rf = auc(fpr_rf, tpr_rf)

fpr_gb, tpr_gb, _ = roc_curve(y_test, y_prob_gb)
auc_gb = auc(fpr_gb, tpr_gb)

# Graficar las Curvas ROC
plt.figure(figsize=(10, 8))
plt.plot(fpr_tree, tpr_tree, label=f'Árbol de Decisión (AUC = {auc_tree:.3f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.3f})')
plt.plot(fpr_gb, tpr_gb, label=f'Gradient Boosting (AUC = {auc_gb:.3f})')
plt.plot([0, 1], [0, 1], 'k--', label='Clasificador Aleatorio') # Línea de azar
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos (Recall)')
plt.title('Curvas ROC para Modelos de Riesgo Crediticio')
plt.legend()
plt.show()

**Interpretación de las Curvas ROC y AUC:**

El modelo de **Random Forest (AUC=0.796)** es mejor que el **Gradient Boosting (AUC=0.775)** y mucho mejor que el **Árbol de Decisión (AUC=0.576)** para la tarea de *distinguir* entre clientes buenos y malos.

## 6. Del Análisis ROC a la Decisión de Negocio: Ajustando el Umbral

Ya hemos encontrado el modelo con mejor desempeño para nuestro caso: el Random Forest. Pero el paso final para que un modelo sea verdaderamente útil en un contexto económico es alinear sus predicciones con un objetivo de negocio.

Por defecto, un modelo de clasificación usa un umbral de 0.5 para decidir entre "Riesgo 0" y "Riesgo 1". Este umbral es arbitrario y casi nunca es el ideal.

Para nuestro banco, el costo de no detectar a un cliente de "mal riesgo" (un Falso Negativo) es mucho más alto que el costo de verificar dos veces a un cliente bueno (un Falso Positivo).

Por lo tanto, no buscamos el umbral que da el mejor accuracy, sino el que cumple nuestro objetivo de negocio: **Capturar al menos el 80% de los clientes de mal riesgo (Recall = 0.8)**

En esta sección, usaremos los datos de la Curva ROC para encontrar el umbral de probabilidad exacto que cumple este objetivo. Luego, dejaremos de usar .predict() y aplicaremos este nuevo umbral para crear nuestro sistema de decisión final.

In [ ]:
# --- Encontrar el umbral óptimo para Random Forest ---

# Obtener las probabilidades de predicción para la clase positiva (riesgo=1)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

# Calcular la curva ROC
# (tpr = True Positive Rate = Recall)
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_prob_rf)

# Definir nuestro objetivo de negocio
objetivo_recall = 0.80 # Por ejemplo, queremos un recall del 80%

# Encontrar el umbral que cumple nuestro objetivo
try:
    indice_objetivo_rf = np.argmax(tpr_rf >= objetivo_recall)
    umbral_elegido_rf = thresholds_rf[indice_objetivo_rf]
    costo_fpr_rf = fpr_rf[indice_objetivo_rf]
    recall_obtenido_rf = tpr_rf[indice_objetivo_rf]

    print(f"--- Decisión de Negocio (Random Forest) ---")
    print(f"Para alcanzar un Recall de al menos {objetivo_recall*100:.0f}% (obtuvimos {recall_obtenido_rf*100:.1f}%):")
    print(f"Debes usar un umbral de decisión de: {umbral_elegido_rf:.4f}")
    print(f"El costo será una Tasa de Falsos Positivos de: {costo_fpr_rf*100:.1f}%\n")

except ValueError:
    print("El modelo Random Forest no alcanza el recall objetivo.")
    umbral_elegido_rf = 0.5 # Usar default si falla


# --- Aplicar el nuevo umbral a las predicciones ---

if 'umbral_elegido_rf' in locals():
    # Usamos el umbral que acabamos de encontrar
    predicciones_rf_umbral = (y_prob_rf >= umbral_elegido_rf).astype(int)

    # Evaluar el rendimiento con el nuevo umbral
    print(f"--- Reporte de Clasificación (Umbral RF = {umbral_elegido_rf:.4f}) ---")
    print(classification_report(y_test, predicciones_rf_umbral))

    # Visualizar la matriz de confusión
    cm_rf_umbral = confusion_matrix(y_test, predicciones_rf_umbral)
    sns.heatmap(cm_rf_umbral, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Predicción 0 (Bueno)', 'Predicción 1 (Malo)'],
                yticklabels=['Real 0 (Bueno)', 'Real 1 (Malo)'])
    plt.ylabel('Valor Real')
    plt.xlabel('Valor Predicho')
    plt.title('Matriz de Confusión RF con Umbral Personalizado')
    plt.show()

Esto tiene un trade-off (costo-beneficio): estás aceptando clasificar incorrectamente a más clientes buenos (más Falsos Positivos, bajando el recall de la clase 0) a cambio de capturar muchos más clientes malos (más Verdaderos Positivos, subiendo el recall de la clase 1).

## 7. Variables relevantes

Nuestros modelos de ensamble son muy precisos, pero a menudo se les critica por ser "cajas negras". Si no podemos explicar por qué un modelo toma una decisión, ¿cómo podemos confiar en él para decisiones económicas críticas?

Afortunadamente, **los modelos basados en árboles nos ofrecen una poderosa herramienta para ganar interpretabilidad: la importancia de variables (atributo .feature_importances_).**

Este atributo nos muestra qué variables fueron las más consultadas y decisivas para el "comité" de árboles al momento de clasificar el riesgo. Para el banco, esto es muy útil, ya que nos dice qué factores (ej. status_cuenta, duracion_mes) son los verdaderos impulsores del riesgo.

El siguiente código extrae estas importancias de nuestro Random Forest, las conecta con los nombres de las variables ya procesadas (get_feature_names_out()) y grafica las 15 más relevantes.

In [ ]:
# Obtener los nombres de las variables
feature_names = preprocessor.get_feature_names_out()

# Obtener las importancias del modelo
rf_importances = rf_model.feature_importances_

# Crear un DataFrame para facilitar la visualización
rf_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': rf_importances
}).sort_values(by='Importancia', ascending=False)

# Graficar las 15 variables más importantes
plt.figure(figsize=(12, 8))
sns.barplot(
    x='Importancia',
    y='Variable',
    data=rf_importance_df.head(15),
    palette='viridis'
)
plt.title('Top 15 Variables más Importantes - Random Forest', fontsize=16)
plt.xlabel('Importancia (Gini)', fontsize=12)
plt.ylabel('Variable', fontsize=12)
plt.show()